In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import os
import numpy as np
import pandas as pd

In [3]:
# manually add a2e-mmc repos to PYTHONPATH if needed
import sys
module_path = os.path.join(os.environ['HOME'],'a2e-mmc')
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
# wrapper around reader functions to return a single combined dataframe
from mmctools.dataloaders import read_dir, read_date_dirs, read_files

In [5]:
# more sophisticated data readers for when default pd.read_csv() doesn't cut it
from mmctools.measurements.radar import profiler # see mmctools.measurements.*

# Example: Data processing
written by [Eliot Quon](mailto:eliot.quon@nrel.gov)
- Download and combine a series of data files into one dataframe

In [6]:
# dataset name format: project/class.instance.level
dataset = 'wfip2/radar.z04.b0' 
startdate = pd.to_datetime('2016-11-21')
enddate = pd.to_datetime('2016-11-22')

# optional dataset file specs
dataext = 'txt' # file type, dictated by extension
dataext1 = 'winds' # e.g., *.winds.txt

download_path = 'data'
overwrite_files = False # force download even if files already exist

## Download data from the DAP
This depends on the `a2e` module provided by the `dap-py` package (https://github.com/a2edap/dap-py). Current, the module is in beta, located within a private repo--contact [Matt Macduff](mailto:Matt.Macduff@pnnl.gov) for more information. 

In [7]:
datapath = os.path.join(download_path, dataset.replace('/','.'))
print('Data path:',datapath)

Data path: data/wfip2.radar.z04.b0


In [8]:
try:
    import A2e
except ImportError:
    print('dap-py package not available; need to manually download files')
else:
    a2e = A2e.A2e()
    a2e.setup_cert_auth()
    filter_arg = {
        'Dataset': dataset,
        'date_time': {
            'between': [startdate.strftime('%Y%m%d%H%M%S'), enddate.strftime('%Y%m%d%H%M%S')]
        }
    }
    if dataext:
        filter_arg['file_type'] = dataext
    if dataext1:
        filter_arg['ext1'] = dataext1
    datafiles = a2e.search(filter_arg)
    print(len(datafiles),'data files selected')
    filelist = a2e.download_files(datafiles, path=download_path, force=overwrite_files)
    if filelist is None:
        print('No files were downloaded; need to manually download files to '+datapath)
    else:
        filelist.sort()
        print('Files to process:\n ','\n  '.join(filelist[:5]),'\n  ...')

No authentication found. Using guest credentials...
username: quon
password: ········
Success!
25 data files selected
Download successful! data/wfip2.radar.z04.b0/radar.z04.b0.20161121.000000.winds.txt
Download successful! data/wfip2.radar.z04.b0/radar.z04.b0.20161121.050000.winds.txt
Download successful! data/wfip2.radar.z04.b0/radar.z04.b0.20161121.030000.winds.txt
Download successful! data/wfip2.radar.z04.b0/radar.z04.b0.20161121.010000.winds.txt
Download successful! data/wfip2.radar.z04.b0/radar.z04.b0.20161121.040000.winds.txt
Download successful! data/wfip2.radar.z04.b0/radar.z04.b0.20161121.020000.winds.txt
Download successful! data/wfip2.radar.z04.b0/radar.z04.b0.20161121.060000.winds.txt
Download successful! data/wfip2.radar.z04.b0/radar.z04.b0.20161121.070000.winds.txt
Download successful! data/wfip2.radar.z04.b0/radar.z04.b0.20161121.080000.winds.txt
Download successful! data/wfip2.radar.z04.b0/radar.z04.b0.20161121.090000.winds.txt
Download successful! data/wfip2.radar.z04.

## Process the downloaded files

### read a single directory

In [9]:
%%time
df = read_dir(datapath,
              reader=profiler, file_filter='*.winds.txt',
             )

CPU times: user 259 ms, sys: 5.09 ms, total: 264 ms
Wall time: 271 ms


In [10]:
df.head()

SPD    DIR  MET_QC  RAD.0  RAD.1  RAD.2  CNT.0  \
datetime            height                                                   
2016-11-21 00:00:12 0.081   3.1  258.0     2.0    0.2    0.8    0.9    7.0   
                    0.138   3.6  279.0     2.0    0.2    0.5    1.4    9.0   
                    0.196   5.6  271.0     2.0    0.1    1.0    2.0    9.0   
                    0.253   6.1  276.0     2.0    0.1    0.9    2.3    9.0   
                    0.310   6.4  280.0     0.0    0.1    0.8    2.5    9.0   

                            CNT.1  CNT.2  SNR.0  SNR.1  SNR.2  QC.0  QC.1  \
datetime            height                                                  
2016-11-21 00:00:12 0.081     7.0    9.0   -6.0    2.0    4.0   0.0   0.0   
                    0.138     9.0    9.0   -7.0   -3.0   -2.0   0.0   0.0   
                    0.196     9.0    9.0   -7.0   -6.0   -1.0   0.0   0.7   
                    0.253     9.0    9.0   -7.0   -7.0   -5.0   0.0   0.0   
                    0.310     9.0    9.0  -11.0   -8.0   -7.0   0.0   0.0   

                            QC.2  
datetime            height        
2016-11-21 00:00:12 0.081    0.0  
                    0.138    0.0  
                    0.196    0.0  
                    0.253    0.0  
                    0.310    0.0

Read in scan properties for each data block
- Tip: Use command `profiler?` to pop up help in a separate window
- Tip 2: Type `profiler(` and press shift+tab 1, 2, or 4 times for quick help

In [11]:
help(profiler)

Help on function profiler in module mmctools.measurements.radar:

profiler(fname, scans=None, check_na=['SPD', 'DIR'], na_values=999999, height_name='HT', read_scan_properties=False, verbose=False)
    Wind Profiler radar with RASS
    
    Users:
    - Earth Sciences Research Laboratory (ESRL)
    - Texas Tech University (TTU)
    
    Assumed data format for consensus data format rev 5.1 based on
    provided reference for rev 4.1 from:
    https://a2e.energy.gov/data/wfip2/attach/915mhz-cns-winds-data-format.txt
    - Winds variables of interest: SPD, DIR(, SNR)
    - RASS variables of interest: T, Tc, W
    
    Additional data format reference:
    https://www.esrl.noaa.gov/psd/data/obs/formats/
    
    Usage
    =====
    scans : int, list, or None
        Number of data blocks to read from file; a list of zero-indexed
        scans to read from file; or set to None to read all data
    check_na : list
        Column names from file to check for n/a or nan values
    na_values :

In [12]:
%%time
scaninfo = []
df = read_dir(datapath,
              reader=profiler, file_filter='*.winds.txt',
              # additional reader argument(s):
              read_scan_properties=scaninfo,
             )

CPU times: user 254 ms, sys: 4.07 ms, total: 258 ms
Wall time: 256 ms


In [13]:
scaninfo[0]

{'station': 'WCO',
 'data_format': 'WINDS    rev 5.1',
 'consensus_avg_time_min': 24,
 'num_beams': 3,
 'num_range_gates': 44,
 'beam:reqd_records_for_consensus': [0, 1, 1],
 'beam:tot_num_records': [9, 10, 10],
 'beam:consensus_window_size_m/s': [0.0, 0.0, 0.0],
 'num_coherent_integrations': [160, 160],
 'num_spectral_averages': [50, 50],
 'pulse_width_ns': [417, 417],
 'inner_pulse_period_ms': [25, 25],
 'fullscale_doppler_value_m/s': [20.5, 20.5],
 'vertical_correction_to_obliques': True,
 'delay_to_first_gate_ns': [3792, 3792],
 'num_gates': [44, 44],
 'gate_spacing_ns': [417, 417],
 'beam:azimuth_deg': [298.0, 208.0, 298.0],
 'beam:elevation_deg': [90.0, 66.4, 66.4]}